In [1]:
#import jieba
import os
import pandas as pd
import numpy as np
import csv
import re
#import chardet
#import emoji
#import joblib

In [3]:
#!pip3 install pandarallel
from pandarallel import pandarallel

#并行初始化
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


## 成功分词+去除停用词代码

In [3]:
#读入数据
group_1 = pd.read_csv('D:\my research\group\group_1\group_1_final.csv',encoding='utf-8')
#group_3.columns = ['0','content','date','time','user_id']
#group_3 = group_3.drop(columns='0')
group_1.head()

,Unnamed: 0,content,date,time,user_id,cut_content,del
0,0,#上海疫情##上海民生##就这一刻##返乡# 上海返乡潮开启，安徽、江苏和河南人最多，隔离和...,2022-04-24,2022-04-24 08:33:43,1000074972,# 上海 疫情 ## 上海 民生 ## 就 这 一刻 ## 返乡 # 上海 返乡 潮 开...,疫情 民生 一刻 返乡 返乡 潮 开启 安徽 江苏 河南人 隔离 做 核酸 钱 返乡 潮...
1,1,#上海疫情##上海民生##上海现状# .上海的现状为什么会变成这样我忍不住地继续瞎想外行领导...,2022-04-23,2022-04-23 05:46:14,1000074972,# 上海 疫情 ## 上海 民生 ## 上海 现状 # . 上海 的 现状 为什么 会 ...,疫情 民生 现状 现状 忍不住 瞎 想 外行 领导 内行 内行 闭嘴 同理 衍生 行业 ...
2,2,#上海疫情##上海民生# 无间道，罗生门，可能本来就是人生如戏吧，是我太认真了。算...,2022-04-22,2022-04-22 11:47:31,1000074972,# 上海 疫情 ## 上海 民生 # 无间道 ， 罗生门 ， 可能 ...,疫情 民生 无间道 罗生门 本来 人生 如戏 是我太 算了 干脆 眼...
3,3,#上海民生##上海疫情##上海团长# 还是你牛逼呀，团长。开个车照送，还是封控小区。,2022-04-22,2022-04-22 09:41:33,1000074972,# 上海 民生 ## 上海 疫情 ## 上海 团长 # 还是 你 牛 逼 呀 ， 团长 ...,民生 疫情 团长 牛 逼 团长 开个 车照 送 封控 小区
4,4,明康汇的采购员你成功地把你们老板一个月来的光辉形象給黑了。@明康汇 @市场监管 @江丄孤舟 ...,2022-04-21,2022-04-21 08:54:29,1000074972,明康汇 的 采购员 你 成功 地 把 你们 老板 一个月 来 的 光辉 形象 給黑 了 。 ...,明康汇 采购员 成功 老板 一个月 光辉 形象 給黑 明康汇 市场监管 江 丄 孤 ...


In [38]:
# 定义分词函数
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [39]:
#将分词结果存入新列
from pandarallel import pandarallel
group_3['cut_content'] = group_3.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Administrator\AppData\Local\Temp\jieba.cache
Loading model cost 0.715 seconds.
Prefix dict has been built successfully.


In [40]:
#创建停用词list
def load_stopwords(filepath):
    m_stopwords = [line.strip()for line in open(filepath,'r',encoding='utf-8').readlines()]
    return m_stopwords

#载入停用词表
stop_word_file = 'D:\my research\dictionary\stop_words_cn.txt'
stopwords = load_stopwords(stop_word_file)

In [41]:
#去掉表情符号、英文字母、数字
#在最后去掉NAN
#完善停用词表
stopwords[0]='!'
stopwords.append('##')
new = (['网页','链接','上海','罗一舟','肖战','朱一龙','任嘉伦','邓伦','微博','视频','顾魏','特别','越来越','好像','我要','刚刚','天天'])
stopwords.extend(new)
stopwords

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '......',
 '...................',
 './',
 '.一',
 '.数',
 '.日',
 '/',
 '//',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '://',
 '::',
 ';',
 '<',
 '=',
 '>',
 '>>',
 '?',
 '@',
 'A',
 'Lex',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'exp',
 'sub',
 'sup',
 '|',
 '}',
 '~',
 '~~~~',
 '·',
 '×',
 '×××',
 'Δ',
 'Ψ',
 'γ',
 'μ',
 'φ',
 'φ．',
 'В',
 '—',
 '——',
 '———',
 '‘',
 '’',
 '’‘',
 '“',
 '”',
 '”，',
 '…',
 '……',
 '…………………………………………………③',
 '′∈',
 '′｜',
 '℃',
 'Ⅲ',
 '↑',
 '→',
 '∈［',
 '∪φ∈',
 '≈',
 '①',
 '②',
 '②ｃ',
 '③',
 '③］',
 '④',
 '⑤',
 '⑥',
 '⑦',
 '⑧',
 '⑨',
 '⑩',
 '──',
 '■',
 '▲',
 '',
 '、',
 '。',
 '〈',
 '〉',
 '《',
 '》',
 '》），',
 '」',
 '『',
 '』',
 '【',
 '】',
 '〔',
 '〕',
 '〕〔',
 '㈧',
 '一',
 '一.',
 '一一',
 '一下',
 '一个',
 '一些',
 '一何',
 '一切',
 '一则',
 '一则通过',
 '一天',
 '一定',
 '一方面',
 '一旦',
 '一时',
 '一来',
 '一样',
 '一次',
 '一片',
 '一番',
 '一直',
 '一致',
 '一般',
 '一起',
 '一转眼'

In [43]:
# 定义一个函数对句子进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stop = stopwords # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stop:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr

#对datdaframe某一列的文本进行分词
out = []
for line in group_3['content']:
    line_seg = seg_sentence(line) # 这里的返回值是字符串
    out.append(line_seg)

In [44]:
#去掉英文字母、英文字母
u_emoji = []
for line in out:
    line_u = str(line)
    line_un = emoji.replace_emoji(line_u,'') # 这里的返回值是字符串
    line_u = re.sub('[a-zA-Z]','',line_un) #去掉英文字母
    line_f = re.sub(r'[0-9]+', '', line_u) #去掉数字
    #line_ff = re.sub('([^\u4e00-\u9fa5])', '', line_f) #去掉除中文外全部字符
    u_emoji.append(line_f)

In [45]:
#将分词结果转换成dataframe形式
name = ['del']
test=pd.DataFrame(columns=name,data=u_emoji)

In [46]:
# 将分词结果作为新列，拼接入原始数据框，默认是并集
group_3_del = pd.concat([group_3,test],axis=1)
group_3_del.head()

,content,date,time,user_id,cut_content,del
0,好的，我知道了～拜拜啦您～不懂事的小屁孩,2022-08-01,2022-08-01 16:49:32,1000347570,好 的 ， 我 知道 了 ～ 拜拜 啦 您 ～ 不懂事 的 小屁孩,拜拜 不懂事 小屁孩
1,太开心啦，一次成功！小朋友太爱吃了～为娘很欣慰,2022-07-31,2022-07-31 11:27:09,1000347570,太 开心 啦 ， 一次 成功 ！ 小朋友 太爱 吃 了 ～ 为娘 很 欣慰,太 开心 成功 小朋友 太爱 吃 为娘 欣慰
2,南汇好朋友的再次投喂～🍇葡萄太好吃了吧！,2022-07-30,2022-07-30 20:16:07,1000347570,南汇 好 朋友 的 再次 投 喂 ～ 🍇 葡萄 太 好吃 了 吧 ！,南汇 朋友 投 葡萄 太 好吃
3,周五ootd9.9的F21的T恤真香～我真是省钱小能手包包也是同事送的,2022-07-29,2022-07-29 08:59:32,1000347570,周五 ootd9.9 的 F21 的 T恤 真香 ～ 我 真是 省钱 小 能手 包包 也 是...,周五 . 恤 真香 省钱 能手 包包 同事 送
4,小可爱太乖了～以后妈妈多带你出门哈 上海·上海月星环球港,2022-07-24,2022-07-24 13:32:46,1000347570,小可爱 太乖 了 ～ 以后 妈妈 多带 你 出门 哈 上海 · 上海 月星 环球 港,小可爱 太乖 妈妈 多带 出门 月星 环球 港


In [47]:
group_3_del.shape

(185691, 6)

In [48]:
group_3_del.dropna(axis=0,how='any')
group_3_del.shape

(185691, 6)

In [49]:
#写入csv
group_3_del.to_csv('D:\my research\group\group_3\group_3_final.csv',encoding="utf-8_sig") 

## 词频统计

In [13]:
affect_col_list = ['PA','PE','PD','PH','PG','PB','PK',
                   'NA','NB','NJ','NH','PF','NI','NC',
                   'NG','NE','ND','NN','NK','NL','PC',
                  'MH','MS','MA','MD','ME',
                  'P','N','Ne']
def load_affect_dict(filepath):
    m_affectdict = []
    for m_col in affect_col_list:
        m_col = []
        m_affectdict.append(m_col)
    for m_line in open(filepath,'r',encoding='utf-8').readlines():
        m_line = m_line.strip()
        kwd = m_line.split('\t')[0].strip()
        col = m_line.split('\t')[1].strip()
        m_affectdict[affect_col_list.index(col)].append(kwd)
    return m_affectdict

#载入情感词典
affect_dict_file = 'D:\my research\dictionary\dict_affect.txt'
affect_dict = load_affect_dict(affect_dict_file)

In [7]:
group_3_del = pd.read_csv('D:\my research\group\group_3\group_3_final.csv',encoding='utf-8')
group_2_del.columns = ['0','content','date','time','user_id','cut_content','del']
group_2_del = group_2_del.drop(columns='0')
group_2_del.head()

,content,date,time,user_id,cut_content,del
0,#上海民生##上海疫情##核酸#检测《上海人核酸“续命”尴尬：缺管子、乱加码、排队最长3小时...,2022-06-02,2022-06-02 13:22:28,1000074972,# 上海 民生 ## 上海 疫情 ## 核酸 # 检测 《 上海 人 核酸 “ 续命 ” 尴...,民生 疫情 核酸 检测 核酸 续命 尴尬 缺 管子 乱 加码 排队 最长 小时 核酸 续命 ...
1,#上海民生##上海疫情##上海复工#上海醒来，6月1号不会是哄孩子的吧。6月1日零时起 上海...,2022-05-30,2022-05-30 19:42:39,1000074972,# 上海 民生 ## 上海 疫情 ## 上海 复工 # 上海 醒来 ， 6 月 1 号 不会...,民生 疫情 复工 醒来 月 号 哄 孩子 月 日 零时 有序 恢复 住宅小区 出入 公共...
2,#上海民生##上海疫情# 有关团长《独家｜起底“团长江湖”：平价被抵制、供应商“强制”加价、...,2022-05-16,2022-05-16 09:38:37,1000074972,# 上海 民生 ## 上海 疫情 # 有关 团长 《 独家 ｜ 起底 “ 团长 江湖 ”...,民生 疫情 团长 独家 起底 团长 江湖 平价 抵制 供应商 强制 加价 黑白 团长 大...
3,#居委会##上海民生##上海疫情# 上海奇葩居委干部，假借抗疫强索私吞翡翠，官方属实》一个居...,2022-05-07,2022-05-07 23:39:52,1000074972,# 居委会 ## 上海 民生 ## 上海 疫情 # 上海 奇葩 居委 干部 ， 假借 抗...,居委会 民生 疫情 奇葩 居委 干部 假借 抗疫 强索 私吞 翡翠 官方 属实 居委 干...
4,#上海民生##上海疫情# 祝贺🎉五一劳动节，迎接双满月，你劳动了吗？,2022-05-01,2022-05-01 17:34:27,1000074972,# 上海 民生 ## 上海 疫情 # 祝贺 🎉 五一劳动节 ， 迎接 双 满月 ， 你 ...,民生 疫情 祝贺 五一劳动节 迎接 双 满月 劳动


In [9]:
u_emoji = list(group_3_del['del'])
u_emoji

['民生 疫情 核酸 检测 核酸 续命 尴尬 缺 管子 乱 加码 排队 最长 小时 核酸 续命 尴尬 缺 管子 乱 加码 排队 最长 小时 ',
 '民生 疫情 复工 醒来 月 号 哄 孩子 月 日 零时   有序 恢复 住宅小区 出入 公共交通 运营 机动车 通行 月 日 零时   有序 恢复 住宅小区 出入 公共交通 运营 机动车 通行 ',
 '民生 疫情   团长 独家 起底 团长 江湖 平价 抵制 供应商 强制 加价 黑白 团长 大战 上班族 宅 家当 团长 专业 管理 背景 人士 制定 流程 分组 运作 社区 团购 ',
 '居委会 民生 疫情   奇葩 居委 干部 假借 抗疫 强索 私吞 翡翠 官方 属实 居委 干部 假借 抗疫 物资 名义 盯上 慷慨 捐赠 物资 小富 爱心 人士 盯上 一块 翡翠 反复 居委 名义 纠缠 强行 索要 ',
 '民生 疫情   祝贺  五一劳动节 迎接 双 满月 劳动 ',
 '民生 疫情   真有 佩刀 上岗 志愿者 小区 工作人员 阻止 京东 配送 情况通报 近日 花园 小区 工作人员 阻止 京东 配送 态度 不当 网上 传播 引发 社会 关注 小区 工作人员 阻止 京东 配送 情况通报 ',
 '突如其来 全市 封闭 核酸 打乱 原本 周六 计划 保佑 下周 上班   健健康康 平平安安  ',
 '临时 调整 职场 办公 一大早 收到 同事 爱心 奶茶  久违  ',
 '刚刚 一岁   岁 豆豆 姐姐 ',
 '日常 碎片    居家 办公  班真 不用 去职 场 值得 期待   ',
 '解封 邻居 依然 暖 辛苦 带给 超爱 山姆 瑞士 卷 ',
 '孩子 快乐 真的 简单 ',
 '复工 第一杯 奶茶  聂 总 关爱 ',
 '愿 几年 做 慈善家 未来 安好 加油  ',
 '养娃费 猫  哈哈哈哈 ',
 '感觉 昨天 生 孩子 一岁 哈哈哈哈   宝贝 永远 健康 快乐   平平安安 ',
 '记录  第一次 做 志愿者 团长  加油 加油 杨浦 ',
 '幸福 周一 希望 好运  期待 好消息  ',
 '心存 感激   ',
 '说 喝  元 一杯 一点点 包含 运费 邻居 ',
 '大哥 记得 五年 前 ',
 '第一顿 小龙虾  说 小区 邻居 太棒了 ',
 '快乐 奶茶  居家 办公 

In [50]:
aaa = []
for line in u_emoji:
    line_u = str(line)
    line_uu = line_u.split()
    aaa.append(line_uu)

new=sum(aaa,[])

In [11]:
def feature_extraction(words):
    count = []       
    #切词后词的总数
    cnt_tags = len(words)
    #print(cnt_tags)
     
    #提取每个情感词类的词频比率
    idx = 0
    for g_col in affect_col_list:
        
        #切词后，其中包含情感词的总数
        affect_cnt = 0
        
        #统计每个词类下关键词出现的总频次affect_cnt
        for i in range(cnt_tags):
            s = words[i]
            if(s in affect_dict[idx]):
                affect_cnt += 1
        
        #计算比率
        r_affect = 0.0
        if (cnt_tags > 0):
            r_affect = affect_cnt/cnt_tags
        
        new = str(affect_col_list[idx])+' '+str(affect_cnt)+' '+str(r_affect)
        count.append(new)        
        idx += 1
    
    return count

In [19]:
#小tips：可能出现问题的结构
#b = ['活该 含笑 勤快']
#c = ['活该','含笑','勤快']

In [51]:
affect_cun = feature_extraction(new) # 这里的返回值是字符串
affect_cun

['PA 39199 0.0159015345372292',
 'PE 5523 0.0022404697887475926',
 'PD 5602 0.002272517066189392',
 'PH 70405 0.028560614788479856',
 'PG 9486 0.0038481072634545827',
 'PB 25375 0.01029366664665402',
 'PK 6897 0.0027978490191910457',
 'NA 2211 0.0008969181066306223',
 'NB 10341 0.004194948050957605',
 'NJ 3713 0.0015062220397645864',
 'NH 2486 0.0010084750850672668',
 'PF 3836 0.0015561184337562492',
 'NI 2222 0.0009013803857680881',
 'NC 7801 0.003164567231942779',
 'NG 673 0.00027301035086495196',
 'NE 13078 0.005305244232707046',
 'ND 10290 0.004174259302229355',
 'NN 42109 0.017082010199958784',
 'NK 211 8.55946270913891e-05',
 'NL 1085 0.000440142987650034',
 'PC 2053 0.0008328235517470229',
 'MH 39267 0.015929119535533533',
 'MS 10914 0.004427392227845595',
 'MA 3951 0.0016027695338297551',
 'MD 4613 0.0018713176055572412',
 'ME 7076 0.002870462470609807',
 'P 0 0.0',
 'N 0 0.0',
 'Ne 0 0.0']

In [52]:
count = pd.DataFrame(columns=['name'],data=affect_cun)
count

,name
0,PA 39199 0.0159015345372292
1,PE 5523 0.0022404697887475926
2,PD 5602 0.002272517066189392
3,PH 70405 0.028560614788479856
4,PG 9486 0.0038481072634545827
5,PB 25375 0.01029366664665402
6,PK 6897 0.0027978490191910457
7,NA 2211 0.0008969181066306223
8,NB 10341 0.004194948050957605
9,NJ 3713 0.0015062220397645864


In [53]:
count['class'] = count['name'].map(lambda x:x.split(' ')[0])
count['count'] = count['name'].map(lambda x:x.split(' ')[1])
count['item'] = count['name'].map(lambda x:x.split(' ')[2])

In [54]:
count

,name,class,count,item
0,PA 39199 0.0159015345372292,PA,39199,0.0159015345372292
1,PE 5523 0.0022404697887475926,PE,5523,0.0022404697887475926
2,PD 5602 0.002272517066189392,PD,5602,0.002272517066189392
3,PH 70405 0.028560614788479856,PH,70405,0.028560614788479856
4,PG 9486 0.0038481072634545827,PG,9486,0.0038481072634545827
5,PB 25375 0.01029366664665402,PB,25375,0.01029366664665402
6,PK 6897 0.0027978490191910457,PK,6897,0.0027978490191910457
7,NA 2211 0.0008969181066306223,NA,2211,0.0008969181066306223
8,NB 10341 0.004194948050957605,NB,10341,0.004194948050957605
9,NJ 3713 0.0015062220397645864,NJ,3713,0.0015062220397645864


In [55]:
#写入csv
count.to_csv('D:\my research\group\group_3\group_3_count.csv',encoding="utf-8_sig")

## 主题建模

In [2]:
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
 
# spacy for lemmatization
import spacy
 
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
#读入数据
group_1_del = pd.read_csv('D:\my research\group\group_1\group_1_final.csv',encoding='utf-8')
group_1_del.columns = ['0','content','date','time','user_id','cut_content','del']
group_1_del = group_1_del.drop(columns='0')
group_1_del.head()

,content,date,time,user_id,cut_content,del
0,#上海疫情##上海民生##就这一刻##返乡# 上海返乡潮开启，安徽、江苏和河南人最多，隔离和...,2022-04-24,2022-04-24 08:33:43,1000074972,# 上海 疫情 ## 上海 民生 ## 就 这 一刻 ## 返乡 # 上海 返乡 潮 开...,疫情 民生 一刻 返乡 返乡 潮 开启 安徽 江苏 河南人 隔离 做 核酸 钱 返乡 潮...
1,#上海疫情##上海民生##上海现状# .上海的现状为什么会变成这样我忍不住地继续瞎想外行领导...,2022-04-23,2022-04-23 05:46:14,1000074972,# 上海 疫情 ## 上海 民生 ## 上海 现状 # . 上海 的 现状 为什么 会 ...,疫情 民生 现状 现状 忍不住 瞎 想 外行 领导 内行 内行 闭嘴 同理 衍生 行业 ...
2,#上海疫情##上海民生# 无间道，罗生门，可能本来就是人生如戏吧，是我太认真了。算...,2022-04-22,2022-04-22 11:47:31,1000074972,# 上海 疫情 ## 上海 民生 # 无间道 ， 罗生门 ， 可能 ...,疫情 民生 无间道 罗生门 本来 人生 如戏 是我太 算了 干脆 眼...
3,#上海民生##上海疫情##上海团长# 还是你牛逼呀，团长。开个车照送，还是封控小区。,2022-04-22,2022-04-22 09:41:33,1000074972,# 上海 民生 ## 上海 疫情 ## 上海 团长 # 还是 你 牛 逼 呀 ， 团长 ...,民生 疫情 团长 牛 逼 团长 开个 车照 送 封控 小区
4,明康汇的采购员你成功地把你们老板一个月来的光辉形象給黑了。@明康汇 @市场监管 @江丄孤舟 ...,2022-04-21,2022-04-21 08:54:29,1000074972,明康汇 的 采购员 你 成功 地 把 你们 老板 一个月 来 的 光辉 形象 給黑 了 。 ...,明康汇 采购员 成功 老板 一个月 光辉 形象 給黑 明康汇 市场监管 江 丄 孤 ...


In [5]:
# 构造 TF-IDF
#tf_idf_vectorizer = TfidfVectorizer()
tf_idf_vectorizer = TfidfVectorizer(min_df = 0.001,max_df = 0.8)
tf_idf = tf_idf_vectorizer.fit_transform(group_1_del['del'].values.astype('U'))

In [6]:
# 特征词列表
feature_names = tf_idf_vectorizer.get_feature_names()
# 特征词 TF-IDF 矩阵
matrix = tf_idf.toarray()
feature_names_df = pd.DataFrame(matrix,columns=feature_names)
print(feature_names_df)
feature_names_df

        nan  一下子  一个个  一个多月       一个月   一人   一件   一份   一会  一会儿  ...   高价   高考  \
0       0.0  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1       0.0  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2       0.0  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3       0.0  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4       0.0  0.0  0.0   0.0  0.544501  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...     ...  ...  ...   ...       ...  ...  ...  ...  ...  ...  ...  ...  ...   
284768  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
284769  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
284770  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
284771  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
284772  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

         魔幻   鸡蛋   麻烦   黄瓜 

,nan,一下子,一个个,一个多月,一个月,一人,一件,一份,一会,一会儿,...,高价,高考,魔幻,鸡蛋,麻烦,黄瓜,默默,鼓励,鼻子,龚俊
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.544501,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284768,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
284769,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
284770,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
284771,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 进行矩阵运算时涉及的矩阵太大了，导致分配的内存不够用

In [7]:
# 指定 lda 主题数
n_topics = 5
# 要输出的每个主题的前 n_top_words 个主题词数
n_top_words = 20

lda = LatentDirichletAllocation(
    n_components=n_topics, max_iter=50,
    learning_method='online',
    learning_offset=50.,
    random_state=0)
# 核心，给 LDA 喂生成的 TF-IDF 矩阵
lda.fit(tf_idf)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=5, random_state=0)

In [8]:
def top_words_data_frame(model: LatentDirichletAllocation,
                         tf_idf_vectorizer: TfidfVectorizer,
                         n_top_words: int) -> pd.DataFrame:
    '''
    求出每个主题的前 n_top_words 个词

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    tf_idf_vectorizer : sklearn 的 TfidfVectorizer
    n_top_words :前 n_top_words 个主题词

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    rows = []
    feature_names = tf_idf_vectorizer.get_feature_names()
    for topic in model.components_:
        top_words = [feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]]
        rows.append(top_words)
    columns = [f'topic {i+1}' for i in range(n_top_words)]
    df = pd.DataFrame(rows, columns=columns)

    return df


def predict_to_data_frame(model: LatentDirichletAllocation, X: np.ndarray) -> pd.DataFrame:
    '''
    求出文档主题概率分布情况

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    X : 词向量矩阵

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    # 求出给定文档的主题概率分布矩阵
    matrix = model.transform(X)
    columns = [f'P(topic {i+1})' for i in range(len(model.components_))]
    df = pd.DataFrame(matrix, columns=columns)
    return df

In [9]:
# 计算 n_top_words 个主题词
top_words_df = top_words_data_frame(lda, tf_idf_vectorizer, n_top_words)
top_words_df
# 保存 n_top_words 个主题词到 csv 文件中
#top_words_df.to_csv(top_words_csv_path, encoding='utf-8-sig', index=None)

# 转 tf_idf 为数组，以便后面使用它来对文本主题概率分布进行计算
X = tf_idf.toarray()

# 计算完毕主题概率分布情况
predict_df = predict_to_data_frame(lda, X)
predict_df
# 保存文本主题概率分布到 csv 文件中
#predict_df.to_csv(predict_topic_csv_path, encoding='utf-8-sig', index=None)

,P(topic 1),P(topic 2),P(topic 3),P(topic 4),P(topic 5)
0,0.628986,0.052565,0.095567,0.169832,0.053050
1,0.052281,0.052984,0.118067,0.724547,0.052121
2,0.056969,0.603235,0.055682,0.229050,0.055064
3,0.068798,0.068413,0.724369,0.069575,0.068845
4,0.073293,0.075690,0.704058,0.073293,0.073667
...,...,...,...,...,...
284768,0.114168,0.280561,0.337677,0.193695,0.073900
284769,0.058767,0.115330,0.521206,0.165408,0.139290
284770,0.078508,0.146194,0.334482,0.053888,0.386928
284771,0.077170,0.080622,0.349473,0.107589,0.385145


In [10]:
top_words_df

,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,topic 10,topic 11,topic 12,topic 13,topic 14,topic 15,topic 16,topic 17,topic 18,topic 19,topic 20
0,疫情,快乐,哈哈哈,结束,生活,日常,期待,日记,好吃,希望,睡觉,呜呜,辟谣,自由,隔离,永远,宝宝,记录,美好,赶紧
1,喜欢,啊啊啊,开心,谢谢,日子,真的,打卡,好像,咖啡,有没有,孩子,人生,发现,晚安,妈妈,越来越,每日,阳光,特别,时代
2,核酸,疫情,小区,物资,解封,浦东,检测,小时,春天,可爱,封控,抗原,感觉,晚上,封闭,感谢,昨天,早上,隔离,真的
3,真的,世界,快递,中国,终于,东西,疫情,好看,一点,余生,买菜,指教,今日,京东,两个,不到,运动,这是,顾魏,救命
4,新增,无症状,确诊,本土,疫情,居家,隔离,加油,上班,分享,新冠,音乐,办公,我要,在家,老师,停课,真的,不想,防控


In [11]:
import pyLDAvis.sklearn
import pyLDAvis
import pyLDAvis.gensim

In [12]:
# 可视化 html 文件路径
html_path = 'document-lda-visualization.html'
# 使用 pyLDAvis 进行可视化
data = pyLDAvis.sklearn.prepare(lda, tf_idf, tf_idf_vectorizer)
pyLDAvis.save_html(data, html_path)
# 清屏
os.system('clear')
# 浏览器打开 html 文件以查看可视化结果
os.system(f'start {html_path}')

#print('本次生成了文件：',
#      top_words_csv_path,
#      predict_topic_csv_path,
#      html_path)

D:\Program_Files\anaconda\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


0

In [13]:
data = pyLDAvis.sklearn.prepare(lda, tf_idf, tf_idf_vectorizer)
#让可视化可以在notebook内显示
pyLDAvis.display(data)

In [ ]:
pyLDAvis.show(data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [15/Oct/2022 17:10:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 17:10:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 17:12:03] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 17:12:03] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 17:12:03] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 17:12:03] code 404, message Not Found
127.0.0.1 - - [15/Oct/2022 17:12:03] "GET /favicon.ico HTTP/1.1" 404 -


## 生活满意度预测

In [56]:
group_3_del.head()

,content,date,time,user_id,cut_content,del
0,好的，我知道了～拜拜啦您～不懂事的小屁孩,2022-08-01,2022-08-01 16:49:32,1000347570,好 的 ， 我 知道 了 ～ 拜拜 啦 您 ～ 不懂事 的 小屁孩,拜拜 不懂事 小屁孩
1,太开心啦，一次成功！小朋友太爱吃了～为娘很欣慰,2022-07-31,2022-07-31 11:27:09,1000347570,太 开心 啦 ， 一次 成功 ！ 小朋友 太爱 吃 了 ～ 为娘 很 欣慰,太 开心 成功 小朋友 太爱 吃 为娘 欣慰
2,南汇好朋友的再次投喂～🍇葡萄太好吃了吧！,2022-07-30,2022-07-30 20:16:07,1000347570,南汇 好 朋友 的 再次 投 喂 ～ 🍇 葡萄 太 好吃 了 吧 ！,南汇 朋友 投 葡萄 太 好吃
3,周五ootd9.9的F21的T恤真香～我真是省钱小能手包包也是同事送的,2022-07-29,2022-07-29 08:59:32,1000347570,周五 ootd9.9 的 F21 的 T恤 真香 ～ 我 真是 省钱 小 能手 包包 也 是...,周五 . 恤 真香 省钱 能手 包包 同事 送
4,小可爱太乖了～以后妈妈多带你出门哈 上海·上海月星环球港,2022-07-24,2022-07-24 13:32:46,1000347570,小可爱 太乖 了 ～ 以后 妈妈 多带 你 出门 哈 上海 · 上海 月星 环球 港,小可爱 太乖 妈妈 多带 出门 月星 环球 港


In [57]:
a= group_3_del['user_id']
b= list(set(a))
b

[1818394624,
 7463206912,
 7479558149,
 2151317510,
 3665330182,
 3968696326,
 6530596871,
 7547191302,
 2311094287,
 1761509394,
 7162888218,
 3549462557,
 1931345952,
 3359866912,
 6698237986,
 1691025443,
 5124096036,
 5581307941,
 2482831401,
 1746042922,
 1317109803,
 7457177643,
 2934046771,
 2271871033,
 2592211002,
 2848718905,
 3169124415,
 2280816704,
 6316097602,
 1825407043,
 1642856524,
 1710751821,
 5020024911,
 6940917841,
 5855903826,
 1840480341,
 6026887253,
 6474793045,
 5019107417,
 5684822108,
 5503090782,
 5310054500,
 6131023974,
 7187300458,
 2075590763,
 2861924461,
 1544847470,
 5413470320,
 2785771637,
 3835363457,
 3179970691,
 5685510281,
 5581766794,
 5649334411,
 6058836106,
 7472054409,
 7256703119,
 1857978514,
 5280202898,
 5876252821,
 6285754524,
 6073483421,
 7505805468,
 7511867549,
 6219432102,
 2976972972,
 2310406321,
 5182488756,
 1903001781,
 6534135989,
 6854508728,
 1794048190,
 1794179267,
 7478345927,
 3474489544,
 2680258763,
 3612868811,

In [31]:
# 写循环，把id 一致的行写入一个txt中
#模型需要输入的特征是词频比例，需要每个人的所有帖子合在一起
def pick_id(dataframe,id):
    pick = []
    all = []
    id_all = pd.DataFrame(columns=(['id','content']))
    length = len(id)
    for i in range(length):
        u_id = id[i]
        for idx in range(len(dataframe['user_id'])):
            num = dataframe['user_id'][idx]
            if num == u_id:
                pick.append(dataframe['del'][idx])
                
        merge = ""
        for aa in pick:
            merge += (str(aa)+' ')
        all.append(merge)
        pick = []
        i += 1
    id_all['content'] = pd.DataFrame(all)
    id_all['id'] = pd.DataFrame(id)
    return id_all

In [58]:
pi = pick_id(group_3_del,b)
pi

,id,content
0,1818394624,做 两个 颈椎 不行 拼豆 六年 没玩 拼豆 做 蜡笔小新 恐龙 ...
1,7463206912,面包 爱情 选择 情侣 之间 分寸 感有 昨天 做梦 梦到 心宝 想 爸爸妈妈 游乐...
2,7479558149,救命 我友 找到 温馨 可爱 小游戏 验证 老贼 画风 适合 化 梦冰兰 好多 ...
3,2151317510,英格兰 女足 足球 回家 不愧 欧洲 中国队 女足 带来 快乐 永远 大于 男足 年底 世界...
4,3665330182,早 续命 社畜 日常 个性 图标 清凉 泳池 年费 会员 专属 个性 图标 ...
...,...,...
9849,2859433970,跌倒 烧烤 机车 艾 妮 顆拉岛 芭 沙漠 孤狼 金山 事件 宣传科 周六...
9850,3969679346,冰棍 绝配 毛毛 球 夏天 麻花 辫
9851,3155525622,限 户籍 限 地域 岁 接种 新冠 疫苗 超高 补贴 一针 二针 三针 接种 点私 提前 ...
9852,5963448310,论王 律师 干 事 早上 点钟 出门 先 启东市 仲裁委 开庭 赶去 昆山 当事人 认定 工...


In [59]:
#特征提取
def feature_extraction_new(words):
    item = []
    #切词后词的总数
    cnt_tags = len(words)
    #print(cnt_tags)
     
    #提取每个情感词类的词频比率
    idx = 0
    for g_col in affect_col_list:
        
        #切词后，其中包含情感词的总数
        affect_cnt = 0
        
        #统计每个词类下关键词出现的总频次affect_cnt
        for i in range(cnt_tags):
            s = words[i]
            if(s in affect_dict[idx]):
                affect_cnt += 1
        
        #计算比率
        r_affect = 0.0
        if (cnt_tags > 0):
            r_affect = affect_cnt/cnt_tags
         
        item.append(r_affect)
        idx += 1
    
    return item

In [60]:
f =  pi['content'][1]
len(f)

521

In [61]:
mod_file = 'D:/ww/BDP-Resource/data/swls.mod'
clf = joblib.load(mod_file)
score = []
#word = []
for i in range(len(pi['content'])):
    uu = pi['content'][i]
    word = uu.split(' ')
    item_feature = feature_extraction_new(word)
    reshape = np.array(item_feature).reshape(1,-1)
    result = clf.predict(reshape)
    score.append(result)
    
pi['score'] = pd.DataFrame(score)

In [62]:
#写入csv
pi.to_csv('D:\my research\group\group_3\group_3_score.csv',encoding="utf-8_sig")

### unicode 尝试

In [63]:
#编码 / encode：将 Unicode 字符串转换为特定编码格式对应的字节码的过程
#解码 / decode：将特定编码格式的字节码转换为对应的 Unicode 字符串的过程

'美丽人生'.encode('utf-8')

b'\xe7\xbe\x8e\xe4\xb8\xbd\xe4\xba\xba\xe7\x94\x9f'

In [64]:
#'美丽人生'.decode('utf-8')

In [65]:
b'\xe7\xbe\x8e\xe4\xb8\xbd\xe4\xba\xba\xe7\x94\x9f'.decode('utf-8')

'美丽人生'

In [66]:
a= u'美丽人生'

In [67]:
a

'美丽人生'

## 总结
#### str和bytes是同水平的，下面图中’美丽人生‘和b'\xe7\xbe\x8e\xe4\xb8\xbd\xe4\xba\xba\xe7\x94\x9f'，
#### 其实是str与bytes的相互转换。开头的b表示这是一个bytes类型。\xe7是十六进制的表示方式，它占用1个字节的长度，
#### 因此”中文“被编码成utf-8后，我们可以数得出一共用了12个字节，每个汉字占用3个。
#### 字符串类str有encode()方法，是从字符串向比特流的编码过程。而bytes类型有decode()方法，是从比特流向字符串解码的过程。

#### 所以用unicode处理数据，可以理解为还是用str的形式，编码为unicode，所以在输出窗口中看到的依然会是中文。
#### 但是如果想把dataframe中的数据都转成unicode是不是只能通过写循环了，最好是自己定义一个函数